In [8]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-

# File data.py

import nltk
from nltk.corpus import wordnet as wn

## variables globales
vocab, vect_dict, embed_dict = [], {}, {}

def read_data(lang,type='embeds'):
    vocab.clear()
    vect_dict.clear()
    embed_dict.clear()

    vectors = {}

    if lang == 'fra':
        if type != 'embeds':
            file = "rg65_french.txt"
        else:
            file = "vecs100-linear-frwiki"
    else:
        if type != 'embeds':
            file = "datasets/ws353.txt"
        else:
            file = "vectors_datatxt_250_sg_w10_i5_c500_gensim_clean"
    with open(file, encoding='utf-8') as f:
        for line in f:
            l = line.split(" ")
            if (len(l) == 3):                                                   ## retrait du saut de ligne '\n' pour le fichier de similarité
                n = l[2].split('\n')
                l[2] = float(n[0])
                vector = [l[1],l[2]]
            else :
                del l[len(l)-1]
                vector = [float(val) for val in l[2:len(l)]]                        ## récupération des vecteurs du type [mot 2, valeur] ou du type [valeur1, valeur2,..., valeurX] pour les fichiers d'embeddings
            #print(len(vector))
            if len(vector) == 2:
              if l[0] not in vectors.keys():                                    ## ajout des vecteurs dans un dictionnaire avec le mot 1 en clé et le vecteur en valeur
                  vocab.append(l[0])
                  vectors[l[0]] = vector
                  if l[1] not in vectors.keys():                                ## ajout des vecteurs dans un dictionnaire avec le mot 2 en clé et le vecteur [mot 1, valeur] en valeur
                    vocab.append(l[1])
                    vectors[l[1]] = [[l[0],float(l[2])]]
                  else :
                    vectors[l[1]].append([l[0],float(l[2])])
              else :
                  vectors[l[0]].append(vector)
                  if l[1] not in vectors.keys():                                ## ajout des vecteurs dans un dictionnaire avec le mot 2 en clé et le vecteur [mot 1, valeur] en valeur
                    vectors[l[1]] = [[l[0],float(l[2])]]
                  else :
                    vectors[l[1]].append([l[0],float(l[2])])
            else :
              if l[0] not in vectors.keys():                                    ## ajout des vecteurs dans un dictionnaire avec le mot 1 en clé et le vecteur en valeur
                  vocab.append(l[0])
                  vectors[l[0]] = vector
              else :
                  vectors[l[0]].append(vector)

    return vectors,vocab



similarity_dict, similarity_vocab = read_data("fra","vect")                          ## dictionnaire pour la similarité cosinus

#print(vect_dict['corde'])
#print(vect_dict)
#print("VECTS",len(vect_dict))

embeddings_dict, embeddings_vocab = read_data("fra")                ## dictionnaire pour le retrofitting et la tâche d'analyse de sentiments
#print("corde : ", embed_dict['corde'])
#print("la : ", embed_dict['la'])
#print("EMBEDS",len(embed_dict))

def find_vector(word,dic):
    ## Fonction qui permet de récupérer le vecteur d'un certain mot
    ## return [mot 2, value] pour le cas du dictionnaire vect_dict
    ## return [embeddings] pour le cas du dictionnaire embed_dict
    for key, value in dic.items():
        if word == key:
            return value
    return("Le mot n'a pas été trouvé dans le lexique.")

#print(find_vector("corde", embed_dict))
#print(find_vector("idk", embed_dict))


vocabulary = set(embeddings_vocab + similarity_vocab)
vocabulary = list(vocabulary)
print(len(vocabulary))

17481
17481


In [24]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-

## File retrofitting.py
import nltk
from nltk.corpus import wordnet as wn
nltk.download('omw')
nltk.download('wordnet')  # utilisation de WOLF via NLTK wordnet

#from data import similarity_dict, embeddings_dict, vocabulary

def get_synsets(word,lang):
    ## Méthode pour récupérer tous les mots en relation avec celui donné en argument
    return wn.synsets(word,lang=lang)


def lemma(synsets,lang):
    ## Méthode pour récupérer uniquement les mots des synsets et pas 'word.n.01' par exemple
    lemmas,list_lemmas = [],[]
    for synset in synsets:
        lemmas = synset.lemma_names(lang)
        for lemma in lemmas:
            list_lemmas.append(lemma)
    return list_lemmas


def get_hypernyms(word,lang):
    ## Méthode pour récupérer tous les mots en relation d'hypernymie avec celui donnée en argument
    synsets = get_synsets(word,lang=lang)
    hyp = []
    if synsets != [] :
        for synset in synsets:
            hyp += synset.hypernyms()
    return hyp


def get_hyponyms(word,lang):
    ## Méthode pour récupérer tous les mots en relation d'hyponymie avec celui donnée en argument
    synsets = get_synsets(word,lang=lang)
    hyp = []
    if synsets != [] :
        for synset in synsets:
            hyp += synset.hyponyms()
    return hyp


def neighbors(word,lang,rel='neighb',list_neighb=[]):
    ## Méthode pour récupérer les voisins d'un certain mot en fonction du type de relation donnée en argument ou
    list_neighb = []
    # Récupération des synsets avec le type de relation précisé ou non
    if rel == 'hyponym':
        synsets = get_hyponyms(word,lang)
    elif rel == 'hypernym':
        synsets = get_hypernyms(word,lang)
    elif rel == 'synonym':
        synsets = get_synonyms(word,lang)
    else:
        synsets = get_synsets(word,lang)

    # Ajout de la liste de tous les mots appartenant au synset à un dictionnaire dont la clé est le mot donné en argument et la valeur est une liste de mots voisins
    for synset in synsets:
        if (synset not in list_neighb) or (list_neighb == []):
            if synset in vocab:
                list_neighb.append(synset.lemma_names(lang))
    return list_neighb


def retrofit(num_iter,vocab,word_dict,lang,relation='neighb'):
    ## Fonction de retrofitting
    ## D'après l'algorithme de Faruqui
    vocabulary = vocab.intersection(set(word_dict.keys()))
    vectors_dict = word_dict

    for iter in range(num_iter):

        for word in vocabulary:
            if word in vectors_dict.keys():
                word_vect = vectors_dict[word]
            else : word_vect = []

            list_neighb = neighbors(word,lang,relation)
            num_neighb = len(list_neighb)

            if list_neighb != []:
                word_vect = word_dict[word] * num_neighb
                for neighb in list_neighb:
                    if neighb in vectors_dict.keys():
                        word_vect += vectors_dict[neighb]
                        #print("word vect 2",word_vect)
                vectors_dict[word] = word_vect/(2*num_neighb)

    print("DONE")
    return vectors_dict

"""
BATCH_SIZE = 5
vocab_list = list(vocab)
new_vectors = {}

for i in range(5):
    batch = vocab_list[i:BATCH_SIZE]
    print(batch)
    set_batch = set(batch)
    new_vectors.update(retrofit(5,set_batch,embed_dict,'fra'))
print("TEST AVANT RETROFIT",batch[0],embed_dict[batch[0]])
print("TEST APRES RETROFIT",batch[0],new_vectors[batch[0]])
"""

[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'\nBATCH_SIZE = 5\nvocab_list = list(vocab)\nnew_vectors = {}\n\nfor i in range(5):\n    batch = vocab_list[i:BATCH_SIZE]\n    print(batch)\n    set_batch = set(batch)\n    new_vectors.update(retrofit(5,set_batch,embed_dict,\'fra\'))\nprint("TEST AVANT RETROFIT",batch[0],embed_dict[batch[0]])\nprint("TEST APRES RETROFIT",batch[0],new_vectors[batch[0]])\n'

In [26]:
# File movies_critics.py

try:
  import allocine
except ImportError:
  !pip install allocine-wrapper  ## Pas sûres d'utiliser cette librairie

import re
import csv
import torch
import numpy as np
import sklearn 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from keras import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

embed_size = len(embeddings_dict["and"])  ## récupération du nombre de features d'un mot du vocabulaire pour créer la matrice d'embeddings

def get_data_eng_sentiment(typ):
  ## récupération des données pour l'analyse de sentiments en anglais
  ## fichiers de stanford
  if typ == "train" :
    file = "stanford_raw_train.txt"
  elif typ == "dev":
    file = "stanford_raw_dev.txt"
  else:
    file = "stanford_raw_test.txt"
  critics = []

  with open(file, encoding='utf-8') as f:
    corpus_vocab = []
    for line in f:
      ## chaque ligne de type "1 | -1" + "text"
      if re.match(r'\d\s[A-Za-z]+', line):
        l = line.split(" ")
        #print(l)
        sentiment = l[0]
        text = []
        for i in range(1,len(l)):
          text.append(l[i])
        critics.append((sentiment," ".join(text)))
  return critics

def get_data_fra_sentiment(typ="train"):
  ## récupération des données pour l'analyse de sentiments en français
  ## fichiers de allociné, corpus créé par Théophile Blard
    file_table = []
    if typ == "train" :
        file = "train.csv"
    elif typ == "dev":
        file = "valid.csv"
    else:
        file = "test.csv"
    critics = []

    with open(file, newline='') as f:
      reader = csv.reader(f)
      for row in reader:
          file_table.append(row)
      f.close()

    for i in range(len(file_table)):
      if file_table[2] == 0:
        file_table[2] = -1
      critics.append((file_table[1],file_table[2]))
    return critics


def fit_data(critics):
  ## fonction pour que le modèle soit entraîné sur les données 
  X,Y = [],[]
  for sent,text in critics :
    X.append(text)
    Y.append(sent)

  tokenizer = Tokenizer(lower=True,split=' ')
  tokenizer.fit_on_texts(X)
  X = tokenizer.texts_to_sequences(X)
  X = pad_sequences(X)
  return X,Y

def get_embedding_mat(embed_dict,corpus_vocab):
  ## Fonction de création de matrice d'embeddings
  matrix = np.zeros((len(corpus_vocab),embed_size))
  #print(matrix.shape) # (10538, 99)
  for i,word in enumerate(corpus_vocab):
    vector = [0]*embed_size
    if (word in embed_dict.keys()) and (len(embed_dict[word]) == embed_size):
      ## On ne récupère que les vecteurs des mots qui ont la bonne taille et qui font partie du dictionnaire des valeurs pré entrainées
      vector = embed_dict[word]
      #print(len(vector))
    matrix[i] = vector
  return matrix

embedding_matrix = get_embedding_mat(embeddings_dict,vocabulary)

train_critics_eng = get_data_eng_sentiment("train")
X_train, Y_train = fit_data(train_critics_eng) 

dev_critics_eng = get_data_eng_sentiment("dev")
X_dev, Y_dev = fit_data(dev_critics_eng) 

train_critics_fra = get_data_fra_sentiment("train")
X_train_f, Y_train_f = fit_data(train_critics_fra) 

# MLP avec sklearn
MLP_model = MLPClassifier(hidden_layer_sizes=(100,),activation='tanh',alpha=0.001,solver='adam',max_iter=5000,n_iter_no_change=5)


def fit_and_predict(X,Y):
    MLP_model.fit(X,Y)
    print("preds : ",MLP_model.predict(X))
    print("preds probas: ",MLP_model.predict_proba(X))
    print("score : ",MLP_model.score(X,Y))
    print("accuracy : ",accuracy_score(Y,MLP_model.predict(X)))
    print("loss : ",MLP_model.loss_)

fit_and_predict(X_dev,Y_dev)

# MLP avec Keras
model = Sequential()
model.add(Embedding(len(vocabulary), embed_size, weights=[embedding_matrix])) 
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam')


ValueError: ignored